In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Ridge, RidgeCV
from sklearn.model_selection import cross_val_score, KFold, GridSearchCV, train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import scale
from catboost import CatBoostRegressor
from sklearn.linear_model import LinearRegression
import lightgbm as lgb
import xgboost as xgb
from matplotlib import pyplot as plt
import seaborn as sns
import math
import time
import datetime
from pandas.tseries.offsets import *
from xgboost import XGBRegressor
from sklearn import preprocessing
from sklearn import linear_model

from pylab import rcParams
%matplotlib inline
rcParams['figure.figsize'] = 10, 8
pd.options.display.max_rows = 999

In [2]:
%%time
flat = pd.read_csv('pdd-data/flat.csv',encoding='cp1251', parse_dates=['sale','date_salestart', 'date_settle'])
price = pd.read_csv('pdd-data/price.csv',encoding='cp1251',parse_dates=['datefrom','dateto','date_salestart'])
status = pd.read_csv('pdd-data/status.csv',encoding='cp1251',parse_dates=['datefrom','dateto'])
test = pd.read_csv('pdd-data/test.csv', parse_dates=['date1'],encoding='cp1251')
train = pd.read_csv('pdd-data/train.csv', parse_dates=['date1'],encoding='cp1251')

y = train.value

train = train.drop(['value','id','start_square','plan_s','plan_m','plan_l','vid_0','vid_1','vid_2'], axis=1)
test = test.drop(['id'], axis=1)

columns = ['id_bulk', 'spalen', 'date1', 'price', 'mean_sq', 'mean_fl','month', 'month_cnt', 'objclass', 'roomcount', 'wall', 'area', 'vh_groups', 'kindergarten', 'school',
           'hospital', 'phok', 'trainarea', 'carwash', 'stores', 'wheelchairs', 'conditioner', 'ventilation', 'elevator', 'garbage_col', 'cctv', 'u_parking', 
           'noauto_yard', 'auto_places', 'factory_zone', 'green_zone', 'to_kremlin', 'to_mall', 'to_sadoviy', 'to_bigroad', 'to_autocross', 'to_undeground_walk',
           'to_factory', 'to_park', 'to_park_walk', 'u_station_ring', 'yard_area', 'dollar', 'mortgage', 'deposit1', 'deposit13', 'depostit3']

train.columns = columns
test.columns = columns

train['value'] = y

train = pd.concat([train, pd.get_dummies(train['objclass'],prefix='c')], axis=1)
test = pd.concat([test, pd.get_dummies(test['objclass'],prefix='c')], axis=1)

for i in ['wall', 'vh_groups','carwash', 'stores', 'wheelchairs', 'u_parking', 'noauto_yard']:
    train[i] = train[i].replace('да', 1).replace('нет', 0)
    test[i] = test[i].replace('да', 1).replace('нет', 0)
    
train = train.drop(['garbage_col','elevator','trainarea', 'objclass'], axis=1)
test = test.drop(['garbage_col','elevator','trainarea', 'objclass'], axis=1)

train = pd.concat([train, pd.get_dummies(train['month'],prefix='m')], axis=1).drop(['month'], axis=1)
months = pd.concat([pd.get_dummies(test['month'],prefix='m'), pd.DataFrame(data = np.zeros((test.shape[0],9)),\
                columns = ['m_1','m_2','m_6','m_7','m_8','m_9','m_10','m_11','m_12'])], axis=1).sort_index(axis=1)
test = pd.concat([test, months], axis=1).drop(['month'], axis=1)

train['id_bulk_sp'] = train['id_bulk'].map(str)+'_'+train['spalen'].map(str)
test['id_bulk_sp'] = test['id_bulk'].map(str)+'_'+test['spalen'].map(str)
flat['id_bulk_sp'] = flat['bulk_id'].map(str)+'_'+flat['spalen'].astype(int).map(str)

flat['otdelka'] = flat.apply(lambda x: 1 if x.otdelka not in ['Не производится', None] else 0, axis=1)

train = pd.concat([train, pd.get_dummies(train['spalen'],prefix='sp')], axis=1)
test = pd.concat([test, pd.get_dummies(test['spalen'],prefix='sp')], axis=1)

salestart = pd.to_datetime(pd.concat([test,train],ignore_index=False).reset_index(drop=True).groupby('id_bulk_sp')['date1'].min())
# salestart = pd.to_datetime(flat.reset_index(drop=True).groupby('id_bulk_sp')['date_salestart'].min())
train['dayfromstart'] = train.apply(lambda x: float((x['date1'] - salestart[x['id_bulk_sp']]).days), axis=1)
test['dayfromstart'] = test.apply(lambda x: float((x['date1'] - salestart[x['id_bulk_sp']]).days), axis=1)

flat.date_settle = flat.date_settle.fillna(flat.date_settle.min())
train['daytosettle'] = train.apply(lambda x: float(( flat[flat.id_bulk_sp == x['id_bulk_sp']]['date_settle'].iloc[0] - x['date1']).days), axis=1)
test['daytosettle'] = test.apply(lambda x: float(( flat[flat.id_bulk_sp == x['id_bulk_sp']]['date_settle'].iloc[0] - x['date1']).days), axis=1)

test['flat_count'] = test.apply(lambda x: len(flat[flat.id_bulk_sp == x['id_bulk_sp']]), axis=1)
train['flat_count'] = train.apply(lambda x: len(flat[flat.id_bulk_sp == x['id_bulk_sp']]), axis=1)

test['flat_left'] = test.apply(lambda x: len(flat[(flat.id_bulk_sp == x['id_bulk_sp'])&(flat.sale >= x['date1'])]), axis=1) 
train['flat_left'] = train.apply(lambda x: len(flat[(flat.id_bulk_sp == x['id_bulk_sp'])&(flat.sale >= x['date1'])]), axis=1)

def getasaled(x,df,period):
    try:
        t = df[(df.id_bulk_sp == x['id_bulk_sp'])&(df.date1 == x['date1'] - DateOffset(months=period))].sort_values('date1').iloc[0]['value']
        return(t)
    except:
        return(None)
    
def getlastprice(x,df,period):
    try:
        t = df[(df.id_bulk_sp == x['id_bulk_sp'])&(df.date1 == x['date1'] - DateOffset(months=period))].sort_values('date1').iloc[0]['price']
        return(t)
    except:
        return(None)

##лаги продаж
train['saledlastmonth'] = train.apply(lambda x: getasaled(x,train,1), axis=1)
train['saledlast2month'] = train.apply(lambda x: getasaled(x,train,2), axis=1)
train['saledlast3month'] = train.apply(lambda x: getasaled(x,train,3), axis=1)
train['saledlast4month'] = train.apply(lambda x: getasaled(x,train,4), axis=1)
train['saledlast5month'] = train.apply(lambda x: getasaled(x,train,5), axis=1)
train['saledlast6month'] = train.apply(lambda x: getasaled(x,train,6), axis=1)

test['saledlastmonth'] = test.apply(lambda x: getasaled(x,train,1), axis=1)
test['saledlast2month'] = test.apply(lambda x: getasaled(x,train,2), axis=1)
test['saledlast3month'] = test.apply(lambda x: getasaled(x,train,3), axis=1)
test['saledlast4month'] = test.apply(lambda x: getasaled(x,train,4), axis=1)
test['saledlast5month'] = test.apply(lambda x: getasaled(x,train,5), axis=1)
test['saledlast6month'] = test.apply(lambda x: getasaled(x,train,6), axis=1)

##лаги цен
train['pricelastmonth'] = train.apply(lambda x: getlastprice(x,train,1), axis=1)
train['pricelast2month'] = train.apply(lambda x: getlastprice(x,train,2), axis=1)
train['pricelast3month'] = train.apply(lambda x: getlastprice(x,train,3), axis=1)
train['pricelast4month'] = train.apply(lambda x: getlastprice(x,train,4), axis=1)
train['pricelast5month'] = train.apply(lambda x: getlastprice(x,train,5), axis=1)
train['pricelast6month'] = train.apply(lambda x: getlastprice(x,train,6), axis=1)

test['pricelastmonth'] = test.apply(lambda x: getlastprice(x,pd.concat([train,test]),1), axis=1)
test['pricelast2month'] = test.apply(lambda x: getlastprice(x,pd.concat([train,test]),2), axis=1)
test['pricelast3month'] = test.apply(lambda x: getlastprice(x,pd.concat([train,test]),3), axis=1)
test['pricelast4month'] = test.apply(lambda x: getlastprice(x,pd.concat([train,test]),4), axis=1)
test['pricelast5month'] = test.apply(lambda x: getlastprice(x,pd.concat([train,test]),5), axis=1)
test['pricelast6month'] = test.apply(lambda x: getlastprice(x,pd.concat([train,test]),6), axis=1)

##лаги остатков



train['prevcount'] = train.apply(lambda x: len(train[(train.date1 < x['date1'])&(train.id_bulk_sp == x['id_bulk_sp'])]), axis=1)
test['prevcount'] = test.apply(lambda x: len(pd.concat([train,test])[(pd.concat([train,test]).date1 < x['date1'])\
                                                                     &(pd.concat([train,test]).id_bulk_sp == x['id_bulk_sp'])]), axis=1)

train['otdelka'] = train.apply(lambda x: flat[flat.id_bulk_sp == x['id_bulk_sp']]['otdelka'].mean(), axis=1)
train['balcon'] = train.apply(lambda x: flat[flat.id_bulk_sp == x['id_bulk_sp']]['balcon'].mean(), axis=1)
test['otdelka'] = test.apply(lambda x: flat[flat.id_bulk_sp == x['id_bulk_sp']]['otdelka'].mean(), axis=1)
test['balcon'] = test.apply(lambda x: flat[flat.id_bulk_sp == x['id_bulk_sp']]['balcon'].mean(), axis=1)

trainfull = train
testfull = test

train = train.drop(['id_bulk_sp', 'date1', 'id_bulk', 'value'], axis=1)
test = test.drop(['id_bulk_sp', 'date1', 'id_bulk'], axis=1)

cv = KFold(n_splits=5, shuffle=True, random_state=12)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:45: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:97: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:98: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

/usr/local/lib/python3.6/

CPU times: user 20min 22s, sys: 599 ms, total: 20min 22s
Wall time: 20min 22s


In [3]:
mashatest = pd.read_csv('Masha/feature_stat_count/test.csv')
mashatrain = pd.read_csv('Masha/feature_stat_count/train.csv')
train2 = pd.concat([train, mashatrain], axis=1) 
test2 = pd.concat([test, mashatest], axis=1)

In [4]:
train2.describe().T

,count,mean,std,min,25%,50%,75%,max
spalen,9244.0,1.750865,1.174914,0.000000,1.000000,2.000000,3.000000,4.000000
price,9244.0,115364.030203,36419.764142,30155.088235,87847.619048,103500.000000,138641.284404,263000.000000
mean_sq,9244.0,56.749243,25.131018,18.900000,37.710000,54.590000,77.200000,141.650000
mean_fl,9244.0,10.553765,5.112817,1.000000,7.000000,10.000000,14.000000,32.000000
month_cnt,9244.0,27.921463,8.075706,7.000000,23.000000,30.000000,34.000000,38.000000
roomcount,9244.0,2833.461597,1415.591600,432.000000,1862.000000,2454.000000,3958.000000,5579.000000
wall,9244.0,0.013414,0.115046,0.000000,0.000000,0.000000,0.000000,1.000000
area,9244.0,46.325055,44.395662,1.700000,19.000000,23.250000,70.000000,215.000000
vh_groups,9244.0,0.997404,0.050890,0.000000,1.000000,1.000000,1.000000,1.000000
kindergarten,9244.0,513.190718,281.212265,0.000000,220.000000,470.000000,800.000000,1000.000000


In [7]:
X_train, X_test, y_train, y_test = train_test_split(train2, y, test_size=0.2, random_state=1)
# trainfull1 = X_train
# X_train = X_train.drop(['id_bulk_sp', 'date1', 'id_bulk', 'value'], axis=1).reset_index(drop=True)
# X_test = X_test.drop(['id_bulk_sp', 'date1', 'id_bulk'], axis=1).reset_index(drop=True)

def postproccesing(output):
    output.columns = ['id', 'value']
    output.value = output.value.apply(lambda x: 0 if x<0 else x)
#     output['value'] = output.apply(lambda x: ((X_test.loc[x['id'],'mean_sq']*X_test.loc[x['id'],'flat_left']) 
#                                               if x['value']>(X_test.loc[x['id'],'mean_sq']*X_test.loc[x['id'],'flat_left']) 
#                                               else x['value']), axis=1)
#     df = pd.concat([testfull1, output.value], axis=1)
#     df['max'] = df.mean_sq*df.flat_left
#     merged = pd.concat([df.groupby('id_bulk_sp')['value'].sum(),df.groupby('id_bulk_sp')['max'].mean()], axis=1)

#     koef = merged[merged.value>merged['max']]['max']/merged[merged.value>merged['max']]['value']
#     df['value'] = df.apply(lambda x: x['value'] if x['id_bulk_sp'] not in koef else x['value']*koef[x['id_bulk_sp']], axis=1)

#     output = pd.DataFrame(data = df.value).reset_index()
#     output.columns = ['id', 'value']
#     output.to_csv('result.csv', index=False)
    return(output)

clf = XGBRegressor(n_estimators=2000, learning_rate=0.01, gamma=1, subsample=0.75, 
                   colsample_bytree=0.9, max_depth=9, nthread=-1, reg_alpha=1, reg_lambda=3)
clf.fit(X_train,y_train)
pr = clf.predict(X_test)
output1 = pd.DataFrame(data = pr).reset_index()
# output1 = postproccesing(output)

cat = CatBoostRegressor(logging_level='Silent',n_estimators=3500, l2_leaf_reg=10, learning_rate=0.01)
cat.fit(X_train,y_train)
pr = cat.predict(X_test)
output2 = pd.DataFrame(data = pr).reset_index()
# output2 = postproccesing(output)

gbm = lgb.LGBMRegressor(objective='regression',
                        num_leaves=80,
                        max_depth = 6,
                        learning_rate=0.03,
                        n_estimators=2000,
                       nthread=-1)

gbm.fit(X_train,y_train)
pr = gbm.predict(X_test)
output3 = pd.DataFrame(data = pr).reset_index()

# output3 = postproccesing(output)
# df = pd.DataFrame(clf.feature_importances_,index=list(train.T.index))

# %matplotlib inline
# df[0].sort_values(ascending=False).plot(kind='bar',figsize=(20,10))

comparsion = pd.concat([output1[0], output2[0], output3[0], y_test.reset_index(drop=True)], axis=1)
comparsion.columns = ['xgb', 'cat', 'lgb', 'test']
comparsion['differ_xgb'] = comparsion.xgb - comparsion.test
comparsion['differ_cat'] = comparsion.cat - comparsion.test
comparsion['differ_lgb'] = comparsion.lgb - comparsion.test
comparsion['mean1'] = (comparsion.lgb*0.5+comparsion.xgb*0.5)
comparsion['mean2'] = (comparsion.cat+comparsion.xgb+comparsion.lgb)/3
# comparsion['differ_mean'] = abs(comparsion.mean2 - comparsion.test)

np.sqrt(mean_squared_error(comparsion['mean2'], y_test))

189.64443411770935

In [8]:
np.sqrt(mean_squared_error(comparsion['mean2'], y_test))

189.64443411770935

In [ ]:
comparsion.loc[:,['mean2','test','differ_mean']].sort_values('differ_mean', ascending=False)

In [103]:
from sklearn.feature_selection import RFECV

gbm = lgb.LGBMRegressor(objective='regression',
                        num_leaves=80,
                        max_depth = 6,
                        learning_rate=0.03,
                        n_estimators=2000, nthread=-1)
# clf = XGBRegressor(n_estimators=400, learning_rate=0.08, gamma=1, subsample=0.75, colsample_bytree=1, max_depth=6, nthread=-1)
# train1 = train.fillna(0)
rfecv = RFECV(estimator=gbm, step=1, cv=cv, scoring='neg_mean_squared_error', n_jobs=-1)
rfecv.fit(train1, y)

print("Optimal number of features : %d" % rfecv.n_features_)

Optimal number of features : 67


In [104]:
np.sqrt(abs(rfecv.grid_scores_))

array([309.06000606, 299.73298927, 267.40745136, 257.26557161,
       243.34603692, 237.88662076, 228.89184035, 225.2179408 ,
       223.4124908 , 222.5647934 , 220.45658921, 215.99582419,
       214.8422561 , 213.24831611, 213.52338756, 214.82097166,
       212.12681576, 212.06813053, 210.80674628, 211.07379585,
       210.8382259 , 211.10700737, 210.58673501, 210.15359389,
       209.42146513, 209.76038018, 210.27888672, 209.69068231,
       210.75434351, 209.86404159, 210.85181186, 210.41475531,
       210.41505425, 209.2707632 , 210.42240312, 209.72864849,
       209.14795699, 209.30624742, 209.5912339 , 209.23851801,
       208.80791985, 208.58696431, 208.44754903, 208.47718186,
       208.85018539, 208.25802551, 208.44260822, 208.52279202,
       208.60692869, 208.91552396, 208.89136432, 208.28058018,
       208.94729614, 208.71346671, 208.90466326, 208.1928179 ,
       208.51794935, 209.22585625, 208.46296791, 208.04442815,
       208.43237193, 208.3201479 , 208.59548409, 208.69

In [ ]:
ranks = rfecv.ranking_.tolist()
names = train.T.index.tolist()
ch = pd.DataFrame(data=ranks, columns=['ranks'])
ch['names'] = names
ch.sort_values('ranks', ascending=False)

In [ ]:
laststatus = status.sort_values('datefrom').drop_duplicates(subset='id_flatwork', keep='last') 
laststatus.index = laststatus.id_flatwork
laststatus = laststatus[laststatus.stat_name=='Реализован']

sorteddates = pd.concat([flat, laststatus.datefrom], axis=1).loc[:,['date_settle','date_salestart','sale','datefrom']].dropna()

sorteddates['diffe'] = abs(sorteddates['sale']-sorteddates['datefrom'])

sorteddates[sorteddates.diffe>=pd.Timedelta('2 days')].sort_values('datefrom', ascending=False)

In [ ]:
flat = pd.read_csv('flat.csv', parse_dates=['sale','date_salestart', 'date_settle'],encoding='cp1251')
price = pd.read_csv('price.csv',encoding='cp1251',parse_dates=['datefrom','dateto','date_salestart'])
# fixedsaledate = []
notinstatus = []
counter = 0
maximum = flat.sort_values('sale',ascending=False).iloc[0]['sale']
minimum = flat.sort_values('sale',ascending=True).iloc[0]['sale']
def flatgreatagain(x):
    global counter
    counter+=1
    if counter % 10000 == 0:
        print(str(counter))
    try:
        laststatus = status[status.id_flatwork==x['id_flatwork']].sort_values('dateto').iloc[-1]
    except:
        notinstatus.append(x['id_flatwork'])
        return(x['sale'])
    if (x['sale'].date().strftime("%Y-%m") == laststatus['datefrom'].date().strftime("%Y-%m"))\
       &(laststatus['stat_name']=='Реализован'):
        return(x['sale'])
    else:
        if laststatus['stat_name']=='Статус после покупки':
            return(x['sale'])
        elif laststatus['stat_name']=='Реализован':
#             fixedsaledate.append(x['id_flatwork'])
#             return(laststatus['datefrom'])
            return(x['sale'])
        elif laststatus['stat_name']=='Не реализуется':
            listofstrange.append(x['id_flatwork'])
            return(minimum)
        else:
            return(x['sale'])

fixedsale = flat.apply(flatgreatagain, axis=1)

In [18]:
laststatus = status.sort_values('datefrom').drop_duplicates(subset=['id_flatwork'],keep='last').reset_index(drop=True).drop('stat', axis=1)
flats = flat.loc[:,['id_flatwork','sale']].reset_index(drop=True)
flats.index=flats.id_flatwork
laststatus.index=laststatus.id_flatwork
result = pd.concat([flats, laststatus], axis=1).dropna()
result['differ'] = abs(result.datefrom-result.sale)

In [49]:
errors = result.drop('id_flatwork', axis=1).sort_values(['sale','differ'],ascending=False)\
[(result.stat_name=='Реализован')&(result['differ'] > pd.Timedelta('5 days'))]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


In [50]:
errors.to_csv('errors.csv')

In [246]:
startper = testfull.date1.min()
test_first = testfull[testfull.date1==startper]
test_second = testfull[testfull.date1==startper+DateOffset(months=1)]
test_third = testfull[testfull.date1==startper+DateOffset(months=2)]

In [ ]:
test_first['flat_left'] = test_first.apply(lambda x: len(flat[(flat.id_bulk_sp == x['id_bulk_sp'])&(flat.sale >= x['date1'])]), axis=1)
test_first['saledlastmonth'] = test_first.apply(lambda x: len(flat[(flat.id_bulk_sp == x['id_bulk_sp'])&(flat.sale < x['date1'])&(flat.sale >= x['date1'] - DateOffset(months=1))]), axis=1)
test_first['saledlast2month'] = test_first.apply(lambda x: len(flat[(flat.id_bulk_sp == x['id_bulk_sp'])&(flat.sale < x['date1'])&(flat.sale >= x['date1'] - DateOffset(months=2))]), axis=1)
test_first['saledlast3month'] = test_first.apply(lambda x: len(flat[(flat.id_bulk_sp == x['id_bulk_sp'])&(flat.sale < x['date1'])&(flat.sale >= x['date1'] - DateOffset(months=3))]), axis=1)

In [ ]:
test_second['flat_left'] = test_first['flat_left'] - test_first['value']/test_first['mean_sq']
test_second['saledlastmonth'] = test_first['value']/test_first['mean_sq']

In [185]:
test = pd.read_csv('test.csv', parse_dates=['date1'])
test['id_bulk_sp'] = test['bulk_id'].map(str)+'_'+test['spalen'].map(str)
test.apply(lambda x: x['price'] - test[(test.id_bulk_sp == x['id_bulk_sp'])&(test.date1 == (x['date1'] - DateOffset(months=1)))]['price'].get(0,0))

In [ ]:
train1 = train.fillna(0)
linreg = LinearRegression()
ridge = linear_model.Ridge(alpha = 0.5)
rf = RandomForestRegressor(n_estimators=1000, criterion='mse', n_jobs=-1)
print('Linear: '+str(np.sqrt(abs(np.mean(cross_val_score(linreg, train1, y, cv=cv, scoring='neg_mean_squared_error'))))))
print('RFReg: '+str(np.sqrt(abs(np.mean(cross_val_score(rf, train1, y, cv=cv, scoring='neg_mean_squared_error'))))))
# print('Ridge: '+str(np.sqrt(abs(np.mean(cross_val_score(ridge, train1, y, cv=cv, scoring='neg_mean_squared_error'))))))

Linear: 254.72664730111663


In [32]:
train1 = train.fillna(0)
t = datetime.datetime.now()
xgb = XGBRegressor(n_estimators=500, learning_rate=0.08, gamma=1, subsample=0.75, max_depth=6, nthread=-1, loss_function='RMSE', random_state=59)
print(np.sqrt(abs(np.mean(cross_val_score(xgb, train1, y, cv=cv, scoring='neg_mean_squared_error',n_jobs=-1)))))
print((datetime.datetime.now() - t).total_seconds())

199.4397979564409
18.020291


In [14]:
#valid
from sklearn.model_selection import KFold

kf = KFold(n_splits=5, random_state=42, shuffle=True)
kf.get_n_splits(range(len(train)))

print(kf)

# for i, (train_index, valid_index) in enumerate(kf.split(range(len(df)))):
#     train = df.loc[train_index]
#     valid = df.loc[valid_index]


KFold(n_splits=5, random_state=42, shuffle=True)


In [15]:
import numpy as np

In [21]:
train = train2.sort_index(axis=1)
test = test2.sort_index(axis=1)

def postproccesing(output):
    output.columns = ['id', 'value']
    output.value = output.value.apply(lambda x: 0 if x<0 else x)
    output['value'] = output.apply(lambda x: ((test.loc[x['id'],'mean_sq']*test.loc[x['id'],'flat_left']) 
                                              if x['value']>(test.loc[x['id'],'mean_sq']*test.loc[x['id'],'flat_left']) 
                                              else x['value']), axis=1)
    df = pd.concat([testfull, output.value], axis=1)
    df['max'] = df.mean_sq*df.flat_left
    merged = pd.concat([df.groupby('id_bulk_sp')['value'].sum(),df.groupby('id_bulk_sp')['max'].mean()], axis=1)

    koef = merged[merged.value>merged['max']]['max']/merged[merged.value>merged['max']]['value']
    df['value'] = df.apply(lambda x: x['value'] if x['id_bulk_sp'] not in koef else x['value']*koef[x['id_bulk_sp']], axis=1)

    output = pd.DataFrame(data = df.value).reset_index()
    output.columns = ['id', 'value']
#     output.to_csv('result.csv', index=False)
    return(output)

print('xgb')
clf = XGBRegressor(n_estimators=2000, learning_rate=0.01, gamma=3, subsample=0.75, colsample_bytree=0.9, max_depth=4)
pr = np.zeros(len(test))
for i, (train_index, valid_index) in enumerate(kf.split(range(len(train)))):
    print('i')
    train_cv = train.loc[train_index]
    y_tr_cv = y[train_index]
    valid_cv = train.loc[valid_index]
    y_val_cv = y[valid_index]
    clf.fit(train_cv,y_tr_cv, eval_set=[(valid_cv, y_val_cv)], early_stopping_rounds=100)
    pr += clf.predict(test)
pr = pr/5
output = pd.DataFrame(data = pr).reset_index()
output1 = postproccesing(output)

print('cat')
cat = CatBoostRegressor(logging_level='Silent',n_estimators=3500, l2_leaf_reg=10, learning_rate=0.01)
pr = np.zeros(len(test))
for i, (train_index, valid_index) in enumerate(kf.split(range(len(train)))):
    print('i')
    train_cv = train.loc[train_index]
    y_tr_cv = y[train_index]
    valid_cv = train.loc[valid_index]
    y_val_cv = y[valid_index]
    cat.fit(train_cv,y_tr_cv, eval_set=[(valid_cv, y_val_cv)], early_stopping_rounds=100)
    pr += cat.predict(test)
pr = pr/5
output = pd.DataFrame(data = pr).reset_index()
output2 = postproccesing(output)

print('lgbm')
gbm = lgb.LGBMRegressor(objective='regression',
                        num_leaves=80,
                        max_depth = 6,
                        learning_rate=0.03,
                        n_estimators=2000, reg_lambda=3)
pr = np.zeros(len(test))
for i, (train_index, valid_index) in enumerate(kf.split(range(len(train)))):
    print('i')
    train_cv = train.loc[train_index]
    y_tr_cv = y[train_index]
    valid_cv = train.loc[valid_index]
    y_val_cv = y[valid_index]
    gbm.fit(train_cv,y_tr_cv, eval_set=[(valid_cv, y_val_cv)], early_stopping_rounds=100)
    pr += gbm.predict(test)
pr = pr/5
output = pd.DataFrame(data = pr).reset_index()
output3 = postproccesing(output)

comparsion = pd.concat([output1.value, output2.value, output3.value], axis=1)
comparsion.columns = ['xgb', 'cat', 'lgb']
comparsion['mean1'] = (comparsion.cat*0.5+comparsion.xgb*0.5)
comparsion['mean2'] = (comparsion.cat+comparsion.xgb+comparsion.lgb)/3
output = pd.DataFrame(data = comparsion['mean2']).reset_index()
output.columns = ['id', 'value']
output.to_csv('result_roman.csv', index=False)

xgb
i
[0]	validation_0-rmse:438.494
Will train until validation_0-rmse hasn't improved in 100 rounds.
[1]	validation_0-rmse:435.695
[2]	validation_0-rmse:432.809
[3]	validation_0-rmse:430.05
[4]	validation_0-rmse:427.285
[5]	validation_0-rmse:424.482
[6]	validation_0-rmse:421.883
[7]	validation_0-rmse:419.371
[8]	validation_0-rmse:416.725
[9]	validation_0-rmse:414.237
[10]	validation_0-rmse:411.673
[11]	validation_0-rmse:409.044
[12]	validation_0-rmse:406.48
[13]	validation_0-rmse:403.954
[14]	validation_0-rmse:401.572
[15]	validation_0-rmse:399.27
[16]	validation_0-rmse:396.955
[17]	validation_0-rmse:394.552
[18]	validation_0-rmse:392.357
[19]	validation_0-rmse:390.041
[20]	validation_0-rmse:387.707
[21]	validation_0-rmse:385.405
[22]	validation_0-rmse:383.359
[23]	validation_0-rmse:381.145
[24]	validation_0-rmse:378.95
[25]	validation_0-rmse:376.9
[26]	validation_0-rmse:374.8
[27]	validation_0-rmse:372.72
[28]	validation_0-rmse:370.675
[29]	validation_0-rmse:368.623
[30]	validation_0

[259]	validation_0-rmse:230.055
[260]	validation_0-rmse:229.979
[261]	validation_0-rmse:229.939
[262]	validation_0-rmse:229.824
[263]	validation_0-rmse:229.707
[264]	validation_0-rmse:229.634
[265]	validation_0-rmse:229.512
[266]	validation_0-rmse:229.359
[267]	validation_0-rmse:229.334
[268]	validation_0-rmse:229.258
[269]	validation_0-rmse:229.129
[270]	validation_0-rmse:228.956
[271]	validation_0-rmse:228.914
[272]	validation_0-rmse:228.869
[273]	validation_0-rmse:228.816
[274]	validation_0-rmse:228.815
[275]	validation_0-rmse:228.75
[276]	validation_0-rmse:228.661
[277]	validation_0-rmse:228.575
[278]	validation_0-rmse:228.442
[279]	validation_0-rmse:228.37
[280]	validation_0-rmse:228.278
[281]	validation_0-rmse:228.139
[282]	validation_0-rmse:228.077
[283]	validation_0-rmse:228.014
[284]	validation_0-rmse:227.968
[285]	validation_0-rmse:227.902
[286]	validation_0-rmse:227.828
[287]	validation_0-rmse:227.736
[288]	validation_0-rmse:227.646
[289]	validation_0-rmse:227.575
[290]	vali

[516]	validation_0-rmse:217.553
[517]	validation_0-rmse:217.537
[518]	validation_0-rmse:217.533
[519]	validation_0-rmse:217.53
[520]	validation_0-rmse:217.512
[521]	validation_0-rmse:217.515
[522]	validation_0-rmse:217.522
[523]	validation_0-rmse:217.424
[524]	validation_0-rmse:217.367
[525]	validation_0-rmse:217.385
[526]	validation_0-rmse:217.381
[527]	validation_0-rmse:217.395
[528]	validation_0-rmse:217.388
[529]	validation_0-rmse:217.387
[530]	validation_0-rmse:217.316
[531]	validation_0-rmse:217.281
[532]	validation_0-rmse:217.248
[533]	validation_0-rmse:217.254
[534]	validation_0-rmse:217.257
[535]	validation_0-rmse:217.224
[536]	validation_0-rmse:217.169
[537]	validation_0-rmse:217.189
[538]	validation_0-rmse:217.177
[539]	validation_0-rmse:217.167
[540]	validation_0-rmse:217.131
[541]	validation_0-rmse:217.098
[542]	validation_0-rmse:217.09
[543]	validation_0-rmse:216.996
[544]	validation_0-rmse:216.932
[545]	validation_0-rmse:216.918
[546]	validation_0-rmse:216.869
[547]	vali

[773]	validation_0-rmse:212.937
[774]	validation_0-rmse:212.913
[775]	validation_0-rmse:212.915
[776]	validation_0-rmse:212.932
[777]	validation_0-rmse:212.942
[778]	validation_0-rmse:212.936
[779]	validation_0-rmse:212.928
[780]	validation_0-rmse:212.931
[781]	validation_0-rmse:212.933
[782]	validation_0-rmse:212.913
[783]	validation_0-rmse:212.918
[784]	validation_0-rmse:212.891
[785]	validation_0-rmse:212.849
[786]	validation_0-rmse:212.843
[787]	validation_0-rmse:212.814
[788]	validation_0-rmse:212.827
[789]	validation_0-rmse:212.819
[790]	validation_0-rmse:212.796
[791]	validation_0-rmse:212.802
[792]	validation_0-rmse:212.775
[793]	validation_0-rmse:212.773
[794]	validation_0-rmse:212.751
[795]	validation_0-rmse:212.748
[796]	validation_0-rmse:212.746
[797]	validation_0-rmse:212.729
[798]	validation_0-rmse:212.711
[799]	validation_0-rmse:212.687
[800]	validation_0-rmse:212.69
[801]	validation_0-rmse:212.676
[802]	validation_0-rmse:212.676
[803]	validation_0-rmse:212.665
[804]	val

[1030]	validation_0-rmse:210.16
[1031]	validation_0-rmse:210.153
[1032]	validation_0-rmse:210.154
[1033]	validation_0-rmse:210.132
[1034]	validation_0-rmse:210.031
[1035]	validation_0-rmse:210.02
[1036]	validation_0-rmse:210.024
[1037]	validation_0-rmse:209.983
[1038]	validation_0-rmse:209.96
[1039]	validation_0-rmse:209.862
[1040]	validation_0-rmse:209.855
[1041]	validation_0-rmse:209.857
[1042]	validation_0-rmse:209.857
[1043]	validation_0-rmse:209.829
[1044]	validation_0-rmse:209.819
[1045]	validation_0-rmse:209.819
[1046]	validation_0-rmse:209.811
[1047]	validation_0-rmse:209.813
[1048]	validation_0-rmse:209.79
[1049]	validation_0-rmse:209.778
[1050]	validation_0-rmse:209.772
[1051]	validation_0-rmse:209.766
[1052]	validation_0-rmse:209.771
[1053]	validation_0-rmse:209.765
[1054]	validation_0-rmse:209.771
[1055]	validation_0-rmse:209.77
[1056]	validation_0-rmse:209.775
[1057]	validation_0-rmse:209.78
[1058]	validation_0-rmse:209.784
[1059]	validation_0-rmse:209.789
[1060]	validatio

...

[743]	valid_0's l2: 48841.6
[744]	valid_0's l2: 48843.8
[745]	valid_0's l2: 48819.4
[746]	valid_0's l2: 48802.4
[747]	valid_0's l2: 48797
[748]	valid_0's l2: 48787.8
[749]	valid_0's l2: 48751.8
[750]	valid_0's l2: 48749.6
[751]	valid_0's l2: 48750.6
[752]	valid_0's l2: 48730.7
[753]	valid_0's l2: 48720.5
[754]	valid_0's l2: 48716.2
[755]	valid_0's l2: 48711.3
[756]	valid_0's l2: 48694.9
[757]	valid_0's l2: 48687.9
[758]	valid_0's l2: 48678.8
[759]	valid_0's l2: 48657.6
[760]	valid_0's l2: 48636.4
[761]	valid_0's l2: 48633.6
[762]	valid_0's l2: 48627.2
[763]	valid_0's l2: 48621.2
[764]	valid_0's l2: 48613.9
[765]	valid_0's l2: 48607.3
[766]	valid_0's l2: 48598.7
[767]	valid_0's l2: 48595.5
[768]	valid_0's l2: 48585.9
[769]	valid_0's l2: 48572.1
[770]	valid_0's l2: 48563.9
[771]	valid_0's l2: 48522.7
[772]	valid_0's l2: 48521.2
[773]	valid_0's l2: 48504.4
[774]	valid_0's l2: 48504.9
[775]	valid_0's l2: 48504.7
[776]	valid_0's l2: 48489.6
[777]	valid_0's l2: 48485.5
[778]	valid_0's l2: 48

[1055]	valid_0's l2: 46927.6
[1056]	valid_0's l2: 46934.5
[1057]	valid_0's l2: 46942.5
[1058]	valid_0's l2: 46909.6
[1059]	valid_0's l2: 46903.8
[1060]	valid_0's l2: 46898.4
[1061]	valid_0's l2: 46896.4
[1062]	valid_0's l2: 46891.7
[1063]	valid_0's l2: 46880.7
[1064]	valid_0's l2: 46880.6
[1065]	valid_0's l2: 46874.3
[1066]	valid_0's l2: 46873.3
[1067]	valid_0's l2: 46874.8
[1068]	valid_0's l2: 46876
[1069]	valid_0's l2: 46879.2
[1070]	valid_0's l2: 46874.6
[1071]	valid_0's l2: 46867
[1072]	valid_0's l2: 46868.2
[1073]	valid_0's l2: 46867.8
[1074]	valid_0's l2: 46861.4
[1075]	valid_0's l2: 46846.4
[1076]	valid_0's l2: 46840.7
[1077]	valid_0's l2: 46830
[1078]	valid_0's l2: 46825.2
[1079]	valid_0's l2: 46820.5
[1080]	valid_0's l2: 46810.8
[1081]	valid_0's l2: 46808.3
[1082]	valid_0's l2: 46798.9
[1083]	valid_0's l2: 46786.7
[1084]	valid_0's l2: 46793.8
[1085]	valid_0's l2: 46803.1
[1086]	valid_0's l2: 46802.4
[1087]	valid_0's l2: 46798.2
[1088]	valid_0's l2: 46798.8
[1089]	valid_0's l2:

[1365]	valid_0's l2: 46143
[1366]	valid_0's l2: 46138.3
[1367]	valid_0's l2: 46130.8
[1368]	valid_0's l2: 46128.3
[1369]	valid_0's l2: 46124.8
[1370]	valid_0's l2: 46123.7
[1371]	valid_0's l2: 46120.2
[1372]	valid_0's l2: 46118.3
[1373]	valid_0's l2: 46113.6
[1374]	valid_0's l2: 46114.6
[1375]	valid_0's l2: 46114.4
[1376]	valid_0's l2: 46120.6
[1377]	valid_0's l2: 46127.7
[1378]	valid_0's l2: 46119.3
[1379]	valid_0's l2: 46119.9
[1380]	valid_0's l2: 46122.2
[1381]	valid_0's l2: 46125.5
[1382]	valid_0's l2: 46125
[1383]	valid_0's l2: 46112.7
[1384]	valid_0's l2: 46107.2
[1385]	valid_0's l2: 46099.3
[1386]	valid_0's l2: 46102.9
[1387]	valid_0's l2: 46097.2
[1388]	valid_0's l2: 46097.6
[1389]	valid_0's l2: 46101.1
[1390]	valid_0's l2: 46104.9
[1391]	valid_0's l2: 46106.6
[1392]	valid_0's l2: 46103
[1393]	valid_0's l2: 46103.6
[1394]	valid_0's l2: 46109.8
[1395]	valid_0's l2: 46108.2
[1396]	valid_0's l2: 46109.6
[1397]	valid_0's l2: 46114.2
[1398]	valid_0's l2: 46098.9
[1399]	valid_0's l2:

[1674]	valid_0's l2: 45705.9
[1675]	valid_0's l2: 45701.4
[1676]	valid_0's l2: 45699.2
[1677]	valid_0's l2: 45701.8
[1678]	valid_0's l2: 45698.7
[1679]	valid_0's l2: 45694.5
[1680]	valid_0's l2: 45693.8
[1681]	valid_0's l2: 45688.5
[1682]	valid_0's l2: 45685.7
[1683]	valid_0's l2: 45670.6
[1684]	valid_0's l2: 45663.1
[1685]	valid_0's l2: 45662.9
[1686]	valid_0's l2: 45660.8
[1687]	valid_0's l2: 45661.4
[1688]	valid_0's l2: 45662.6
[1689]	valid_0's l2: 45665.2
[1690]	valid_0's l2: 45663
[1691]	valid_0's l2: 45662.8
[1692]	valid_0's l2: 45659.4
[1693]	valid_0's l2: 45641.8
[1694]	valid_0's l2: 45643.7
[1695]	valid_0's l2: 45641.4
[1696]	valid_0's l2: 45645.4
[1697]	valid_0's l2: 45645.6
[1698]	valid_0's l2: 45645.5
[1699]	valid_0's l2: 45639.3
[1700]	valid_0's l2: 45638.9
[1701]	valid_0's l2: 45635.2
[1702]	valid_0's l2: 45633.4
[1703]	valid_0's l2: 45637.5
[1704]	valid_0's l2: 45633.1
[1705]	valid_0's l2: 45627.9
[1706]	valid_0's l2: 45623.6
[1707]	valid_0's l2: 45621.1
[1708]	valid_0's

[1994]	valid_0's l2: 45371.5
[1995]	valid_0's l2: 45372.1
[1996]	valid_0's l2: 45375.1
[1997]	valid_0's l2: 45374.6
[1998]	valid_0's l2: 45375.6
[1999]	valid_0's l2: 45373.1
[2000]	valid_0's l2: 45368
Did not meet early stopping. Best iteration is:
[2000]	valid_0's l2: 45368


In [22]:
comparsion['mean2'] = (3*comparsion.cat+comparsion.xgb+2*comparsion.lgb)/6
output = pd.DataFrame(data = comparsion['mean2']).reset_index()
output.columns = ['id', 'value']
output.to_csv('result_roman_2.csv', index=False)

In [ ]:
df = pd.DataFrame(clf.feature_importances_,index=list(train.T.index))

%matplotlib inline
df[0].sort_values(ascending=False).plot(kind='bar',figsize=(20,10))

In [62]:
comparsion = pd.concat([output1.value, output2.value, output3.value], axis=1)
comparsion.columns = ['xgb', 'cat', 'lgb']
comparsion['mean1'] = (comparsion.cat*0.5+comparsion.xgb*0.5)
comparsion['mean2'] = (comparsion.cat+comparsion.xgb+comparsion.lgb)/3
output = pd.DataFrame(data = comparsion['mean1']).reset_index()
output.columns = ['id', 'value']
output.to_csv('result.csv', index=False)

In [398]:
train = train.sort_index(axis=1)
test = test.sort_index(axis=1)

In [113]:
output = pd.DataFrame(data = comparsion['mean2']).reset_index()
output.columns = ['id', 'value']
output.to_csv('result.csv', index=False)

In [ ]:
205.92 - LB 187.6409
205.71 - 

In [ ]:
xgb = XGBRegressor()
cat = CatBoostRegressor(logging_level='Silent')
parameters = { 
#               'min_child_weight': [1, 5, 10],
#               'gamma': [0,0.5, 1, 1.5, 2, 5],
#               'subsample': [0.6, 0.8, 1.0],
#               'colsample_bytree': [0.6, 1, 1.2],
#               'learning_rate':[0.02,0.06,0.10],
              'n_estimators':[390,395,400],
              'max_depth': [4,5]
              }
clf = GridSearchCV(xgb, parameters, scoring='neg_mean_squared_error', cv=cv,return_train_score=False, n_jobs=-1)
clf.fit(train, y)
gradresults = pd.DataFrame(clf.cv_results_)
gradresults.sort_values('rank_test_score', axis=0).T

In [ ]:
t = datetime.datetime.now()
gbm = lgb.LGBMRegressor(objective='regression',
                        num_leaves=80,
                        max_depth = 12,
                        learning_rate=0.05,
                        n_estimators=400)                       
print(np.sqrt(abs(np.mean(cross_val_score(gbm, train, y, cv=cv, scoring='neg_mean_squared_error')))))
print((datetime.datetime.now() - t).total_seconds())

t = datetime.datetime.now()
cat = CatBoostRegressor(n_estimators=1500, logging_level='Silent')
print(np.sqrt(abs(np.mean(cross_val_score(cat, train, y, cv=cv, scoring='neg_mean_squared_error')))))
print((datetime.datetime.now() - t).total_seconds())

In [166]:
t = datetime.datetime.now()
cat = CatBoostRegressor(n_estimators=2500, logging_level='Silent',loss_function='RMSE')
print(np.sqrt(abs(np.mean(cross_val_score(cat, train, y, cv=cv, scoring='neg_mean_squared_error')))))
print((datetime.datetime.now() - t).total_seconds())

207.3920863516101
110.311268
